# Objectifs

Ce notebook comporte trois sections principales, la première est un rappel sur les modèles ARIMA, la deuxième traite de l'analyse de Fourier et la dernière est l'analyse spectrale. Les sections 1 et 3 ne contiennent pas d'exercices tandis que la section 2 contient quelques exercices.

# Section 1:
## Rappel : Modèles ARIMA et SARIMAX
Les modèles AR et MA peuvent être combinés avec des différences pour donner la série de modèles **ARIMA(p, d, q)**. La semaine dernière, nous avons introduit la méthode **tsa.statespace.SARIMAX** mise en œuvre dans *statsmodel* : https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html

La fonction SARIMAX prend en entrée le dataframe pandas *df* avec les données, puis deux tuples spécifiant les paramètres *order* et *seasonal_order* pour le modèle.

In [ ]:
import pandas as pd
from pandas import read_excel
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
series = read_excel('BuildingMaterials.xls', sheet_name='Data', header=0, index_col=0, parse_dates=True)
series.index.freq = 'MS'

mod = sm.tsa.statespace.SARIMAX(series, order=(1,1,1), seasonal_order=(0,1,1,12))
results = mod.fit(disp=False)
print(results.summary())

La cellule ci-dessous utilise la bibliothèque python *itertools* pour créer toutes les combinaisons possibles de triplets *p*, *d* et *q* : https://docs.python.org/3/library/itertools.html

L'utilisation d'*itertools* est préférable à la création manuelle des triples avec des boucles for, car elle est conçue pour être rapide et efficace en termes de mémoire.

In [ ]:
import itertools

# Définir les paramètres p, d et q pour prendre n'importe quelle valeur entre 0 et 1
p = d = q = range(0, 2)

# Générer toutes les différentes combinaisons de triplets p, d et q
pdq = list(itertools.product(p, d, q))

# Générer toutes les différentes combinaisons de triplets saisonniers p, d et q (c'est-à-dire, P, D, Q)
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

Voici ci-dessous un exemple de comment sélectionner les meilleurs paramètres de modèle en récupérant directement le score AIC des résultats de l'ajustement SARIMAX.

In [ ]:
import warnings
warnings.filterwarnings("ignore") # spécifie d'ignorer les messages d'avertissement

# Identification du meilleur modèle parmi différentes combinaisons de pdq et de seasonal_pdq
meilleur_score, meilleur_param, meilleur_paramSaisonnal = float("inf"), None, None
for param in pdq:
    for param_saisonnal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(serie, order=param, seasonal_order=param_saisonnal, enforce_invertibility=False)
            results = mod.fit(disp=False)
            if results.aic < meilleur_score:
                meilleur_score, meilleur_param, meilleur_paramSaisonnal = results.aic, param, param_saisonnal
            print('ARIMA{}x{} - AIC:{}'.format(param, param_saisonnal, results.aic))
        except:
            continue # si l'ajustement échoue, continuez simplement à la prochaine combinaison de paramètres

Affichons maintenant le meilleur ensemble de paramètres selon l'AIC.

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
print('Le meilleur modèle est ARIMA{}x{} - AIC:{}'.format(meilleur_param, meilleur_paramSaisonnal, meilleur_score))



# Section 2
## Harmonic analysis and the Fourier transform

Il existe deux types de situations dans lesquelles il est particulièrement utile de considérer une série temporelle d'observations comme une somme de sinusoides :

- Lorsqu'on sait, sur la base de considérations physiques, que le grand signal est vraiment très précisément exprimé comme une somme de quelques sinusoides de fréquence connue. Les marées sont un exemple parfait ; les forces astronomiques se produisent à des fréquences très bien définies et bien connues, donc l'océan répond à ces fréquences.
- Dans la situation opposée, lorsqu'il y a peu ou pas de périodicité perceptible ou d'autre organisation, et lorsqu'un enregistrement d'une période de temps ressemble en caractère, mais totalement différent en spécificités, à celui d'une autre période de temps. Un exemple est la mesure de presque n'importe quelle variable dans un flux turbulent.

Dans le premier de ces cas, on pourrait analyser la série temporelle en utilisant une procédure de moindres carrés pour trouver l'amplitude et la phase de chacune des sinusoides connues.

Dans le deuxième cas, on veut savoir comment l'énergie est répartie entre une gamme de fréquences. Pour cela, la transformation de Fourier est sur mesure. Il vaut la peine de prendre un peu de temps pour comprendre ce que c'est et comment ça fonctionne. Nous nous concentrerons sur la transformée discrète et son inverse ; ce sont ce que nous utilisons en pratique pour l'analyse des données. Les concepts se généralisent facilement aux fonctions continues.

Nous adopterons un point de vue d'algèbre linéaire, et commencerons par une série temporelle triviale : une variable $y$ échantillonnée à seulement deux moments, $t_0$ et $t_1$. La variable peut être tracée comme un vecteur - un point - dans un espace 2-D, avec un axe pour $y_0 = y(t_0)$ et un second pour $y_1 = y(t_1)$. Souvenez-vous, ce n'est pas un espace physique, c'est un espace d'information ; il faut deux informations indépendantes pour spécifier entièrement $y$. Les axes sont perpendiculaires.

Cette spécification de $y$ n'est pas unique ; le même vecteur pourrait être exprimé comme la somme de composantes le long d'axes tournés d'un certain angle par rapport aux originaux. En tournant de 45 degrés CCW, le premier axe devient $(\sqrt{2}/2) [1, 1]\; $ et une réflexion subséquente fait du second axe $(\sqrt{2}/2) [1, -1]\; $, où nous exprimons les vecteurs unitaires des nouveaux axes en termes des vecteurs unitaires des anciens.

Laissons ce premier axe être la première ligne d'une matrice, $M$, et la seconde être la dernière ligne. Alors si $y'$ est $y$ exprimé en termes des axes tournés, nous avons $y' = My\,$. $M$ est orthonormale - son inverse est sa transposée - donc $y = M^{T}y'\:$. Remarquez que $y$ et $y'$ décrivent le même vecteur, mais en termes de sa projection sur différents ensembles de vecteurs unitaires. Le premier axe de $y'$ est proportionnel à la moyenne de $y$ et le second axe est proportionnel à la différence, $y_0 - y_1 $ .

Généralisons maintenant cela. Au lieu de deux temps, il peut y en avoir 3, 4, ou n'importe quel nombre fini. Au-delà de 3, nous ne pouvons plus utiliser l'analogie visuelle de la cartographie de chaque temps à une dimension spatiale, mais les mathématiques restent les mêmes. Si nous avons $N$ points de données, nous pouvons toujours exprimer ces $N$ informations en les projetant sur $N$ axes linéairement indépendants (orthogonaux). Parce qu'ils sont orthogonaux, cette projection est identique à un ajustement des moindres carrés du vecteur $y$ au nouvel ensemble d'axes.

Revenons au cas $N=2\:$ : avec 2 points dans l'enregistrement, si nous imaginons que l'enregistrement est prolongé en répétant la même paire de points encore et encore, nous avons deux périodicités possibles : zéro cycle, ou 1 cycle complet dans la longueur d'enregistrement originale. En utilisant l'espacement des échantillons comme unité de temps, les fréquences sont $\omega_0 = 2 \pi \cdot 0/N\:$ et $\omega_1 = 2 \pi \cdot 1/N\:$ radians par unité de temps. La matrice $M$ a alors des entrées $M_{nk} = \frac{\sqrt{2}}{2}\cos{(2 \pi n k/2)} \; $ .

Pour $N > 2$, devrions-nous continuer ce modèle en utilisant des cosinus ? Nous pourrions, ce qui donnerait une transformée en cosinus ; mais nous pouvons faire mieux en notant que, sauf pour la fréquence zéro et la fréquence $N/2$ cycles par longueur d'enregistrement (en gardant $N$ pair par la suite pour simplifier), il est plus logique de continuer à sélectionner des fréquences avec un nombre entier de cycles par longueur d'enregistrement, et de laisser chaque fréquence avoir une composante en cosinus et une en sinus, donc une amplitude et une phase. Cela donne la transformée de Fourier discrète.

## Transformée de Fourier Discrète

La Transformée de Fourier Discrète et son inverse sont généralement implémentées en utilisant des exponentielles complexes. La version fournie par numpy (https://docs.scipy.org/doc/numpy/reference/routines.fft.html) est typique. Supposons que vous ayez une série de $N$ points, $a_n$. Un exemple typique serait une série temporelle, disons la hauteur de la surface de la mer, ou les vecteurs de courant d'une bouée, échantillonnés régulièrement à intervalles horaires. Alors $\Delta t = 1$ heure, donc si $N = 240$ la longueur de la série serait de $L = 10$ jours ou 240 heures. Le $n$ième temps est $t_n = n \Delta t$.

Les fréquences de Fourier sont un nombre entier de cycles par longueur d'enregistrement : $0, 1, 2...N-1$ cycles par $N \Delta t$, ou $0, 1/(N \Delta t), 2/(N \Delta t), ... (N-1)/(N \Delta t)$ cycles par unité de temps (où $\Delta t$ est l'unité de temps). Définissons la fréquence $k$ comme $f_k = k/(N \Delta t)$ cycles par unité de temps. Alors la DFT mappe la série temporelle sur les amplitudes et phases de fréquence constituantes, $A_k$ comme ceci :

$$A_k = \sum_{n=0}^{N-1} a_n \exp(-i 2 \pi f_k t_n)$$

avec la relation inverse :

$$a_n = \frac{1}{N} \sum_{k=0}^{N-1} A_k \exp(i 2 \pi f_k t_n)$$

Remarquez la quasi-symétrie : les expressions diffèrent par le signe de l'argument de l'exponentielle, et par la normalisation $1/N$ de l'inverse.

Pour relier ces expressions aux formes dans le lien de documentation numpy, évaluez le produit dans l'argument :
$$f_k t_n = (\frac{k}{N \Delta t})(n \Delta t) = \frac{k n}{N}$$

La DFT peut être vue comme la projection - le produit intérieur ou point - du vecteur de la série temporelle sur chacun des vecteurs représentant une exponentielle complexe d'amplitude unitaire à une fréquence de Fourier.

### Variance : Théorème de Parseval

Étant donné la conservation de l'information impliquée dans la TF et son inverse, et l'orthogonalité du nouvel ensemble de vecteurs de base sur lesquels la TF projette les données originales, nous arrivons à un théorème important. Dans sa forme discrète, il stipule que la somme des carrés des échantillons du signal est égale à la somme des amplitudes au carré dans sa TF, multipliée par un facteur de normalisation qui dépend de la façon dont la TF a été définie. Pour la définition utilisée ici, nous avons $$\Sigma |a_k|^2 = \frac{1}{N} \Sigma |A_k|^2$$

Cela signifie que la variance peut être divisée en contributions indépendantes de chaque point de données, ou elle peut être divisée en contributions indépendantes de chaque fréquence de Fourier dans la TF. Ce dernier conduit au concept de *spectre* que nous explorerons dans deux notebooks ultérieurs. D'abord, nous devons bien comprendre comment la DFT fonctionne et ce qu'elle fait.

### Aliasing

Ajouter n'importe quel multiple entier de $2 \pi i$ à l'argument de l'exponentielle dans la DFT ou son inverse n'a aucun effet.

$$\exp\left(-i 2 \pi \frac{(k + NM) n}{N}\right) = 
   \exp\left(-i 2 \pi (\frac{k n}{N} + n M)\right) = 
   \exp\left(-i 2 \pi (\frac{k n}{N})\right)$$
   
Par conséquent, $A_{k + MN}$ = $A_k$ pour tout entier $M$.

### La fréquence Nyquist

La fréquence la plus élevée qui peut être résolue est celle avec deux échantillons par cycle : le pic de la sinusoïde, et la vallée. C'est la fréquence de Nyquist, $f_N = 1 / (2 \Delta t)$ cycles par unité de temps ; la période est l'inverse, $2 \Delta t$.

### Fréquences négatives, ou fréquences supérieures à Nyquist

Si la fréquence la plus élevée que nous pouvons résoudre est la fréquence de Nyquist, cela laisse près de la moitié des fréquences de Fourier trop élevées pour être échantillonnées correctement, n'est-ce pas ? Pas exactement. Ces fréquences de Fourier supérieures à la fréquence de Nyquist sont équivalentes à des fréquences *négatives* inférieures à la fréquence de Nyquist :

$$\exp\left(-i 2 \pi \frac{(N - k) n}{N}\right) = 
   \exp\left(-i 2 \pi (\frac{-k n}{N} + n)\right) = 
   \exp\left(-i 2 \pi (\frac{-k n}{N})\right) = 
   \exp\left(i 2 \pi (\frac{k n}{N})\right)$$
   
Par conséquent, $A_{N - k}$ = $A_{-k}$. La fréquence de Fourier la plus élevée, par exemple, $f_{N-1} = f_{-1}$ qui est juste un cycle par longueur d'enregistrement.

Quelle est la signification des fréquences négatives ? Cela dépend si la série temporelle originale est réelle, comme la hauteur de la surface de la mer, ou complexe, comme la vitesse du courant horizontal, avec les composantes est et nord combinées en un nombre complexe, $u + i v$.

Si elle est réelle, $A_{-k}$ = $A_k^*$. Les fréquences négatives n'apportent aucune nouvelle information.

Si elle est complexe, alors les fréquences positives correspondent à un vecteur tournant dans le sens contraire des aiguilles d'une montre, et les fréquences négatives correspondent à un vecteur tournant dans le sens des aiguilles d'une montre.

## Exemples de DFT

Examinons des exemples de la transformée de Fourier appliquée à des séries temporelles simples. Nous allons préparer les choses avec quelques imports et les valeurs par défaut de matplotlib :

In [ ]:
%matplotlib notebook 

import numpy as np
from numpy.fft import fft, ifft, fftshift, fftfreq
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.rc('lines', markersize=10, markeredgewidth=1.5)
plt.rc('figure', dpi=80, max_open_warning=False)
plt.rc('savefig', dpi=80)

Maintenant, nous créons une fonction pour tracer la série temporelle originale et sa transformation :

In [ ]:
def show_transform(x, y):
    """
    Trace y(x) et sa transformation.
    
    y peut être réel ou complexe.
    """
    x = np.asarray(x)
    y = np.asarray(y)
    fy = fftshift(fft(y))
    freqs = fftshift(fftfreq(len(x), d=(x[1] - x[0])))    
    fig, axs = plt.subplots(nrows=2, constrained_layout=True)

    for ax in axs:
        ax.margins(x=0.05, y=0.1)
        ax.grid(True)
        ax.locator_params(symmetric=True)
        # (La ligne ci-dessus ne semble pas fonctionner comme prévu...)
            
    ax = axs[0]
    if y.dtype.kind == 'c':
        ax.plot(x, y.real, 'r+', x, y.imag, 'bx')
    else:
        ax.plot(x, y, 'k.')
    ax.set_xlabel("Temps")
    
    ax = axs[1]
    ax.plot(freqs, np.abs(fy), 'ko', mfc='none', label='amp')
    ax.plot(freqs, fy.real, 'r+', label='réel')
    ax.plot(freqs, fy.imag, 'bx', label='imaginaire')

    ax.set_xlabel("Fréquence, cycles par unité de temps")
    ax.legend(loc="meilleur", 
                  numpoints=1,
                  fontsize='petit')
    

    return fig, axs

### Transformation d'une sinusoïde

Ensuite, nous créons une fonction pour générer une série temporelle avec une seule sinusoïde, et pour tracer sa série temporelle :

In [ ]:
# Cette fonction génère un signal sinusoïdal
# (ou un signal exponentiel complexe en fonction du paramètre de rotation)
# avec un nombre spécifié de points, de cycles par enregistrement, de décalage de phase et d'intervalle de temps d'échantillonnage.
def sinusoid_pts(npts = 16,
             cycles = 3, # cycles par enregistrement
             phase = 0, # décalage de phase en cycles
             dt = 1, # intervalle de temps d'échantillonnage
             rotation=None # 'positive' ou 'négative' pour l'exponentielle complexe
             ):
    f = cycles / (npts * dt)
    x1 = np.arange(npts, dtype=float) * dt
    phi = 2 * np.pi * (f * x1 - phase)
    if rotation is None:
        y1 = np.cos(phi)
    elif rotation == 'positive':
        y1 = np.exp(1j * phi)
    else:
        y1 = np.exp(-1j * phi)
        
    return x1, y1

# appelle la fonction sinusoid_pts et la fonction show_transform
def sinusoid(*args, **kw):
    x1, y1 = sinusoid_pts(*args, **kw)
    fig, axs = show_transform(x1, y1)
    return fig, axs

Commençons avec les paramètres par défaut, 3 cycles dans un court enregistrement :

In [ ]:
# Commencer avec la configuration par défaut de la fonction sinusoid_pts
npts = 16
sinusoid(npts=npts, cycles=3)

Que se passe-t-il si la fréquence du signal est supérieure à la fréquence de Nyquist ?

In [ ]:
npts = 32
cycles = npts - 3
fig, axs = sinusoid(npts=npts, cycles=cycles)
dt = 0.05  
# Résoudre la courbe aliasée
# npts = 320
x, y = sinusoid_pts(npts=npts/dt, cycles=cycles, dt=dt)
axs[0].plot(x, y, 'r')

Remarquez que nous avons bien une simple courbe cosinus avec 3 cycles dans l'enregistrement de longueur 32 points. La transformation n'a pas de partie imaginaire, et la partie réelle est symétrique par rapport à la fréquence zéro, comme prévu. Parce que nous avons choisi un nombre entier de cycles par longueur d'enregistrement, qui est l'une des fréquences de Fourier, il n'y a qu'une seule fréquence positive avec une amplitude non nulle, et c'est bien sûr la troisième au-dessus de zéro.

Essayez maintenant de lui donner un décalage de phase de 0,25 cycle :

In [ ]:
# Exercice
# appelez la fonction sinusoid et donnez-lui seulement l'argument phase = 0.25

<details>
<summary><b>Cliquez ici pour voir l'indice<b></summary>

fonction(phase=0.25)

</details>

Cela a transformé la courbe d'un cosinus en un sinus. Maintenant, la transformation a une partie *réelle* nulle. Sa partie imaginaire est antisymétrique par rapport à la fréquence zéro, et n'est non nulle que pour 3 cycles par longueur d'enregistrement, comme prévu.

Que se passe-t-il si la série temporelle est un cosinus, mais a un nombre non entier de cycles par longueur d'enregistrement ?

In [ ]:
# Exercice
# appelez la fonction sinusoid et donnez-lui seulement l'argument cycles = 3.5

<details>
<summary><b>Cliquez ici pour voir l'indice<b></summary>

fonction(cycles=3.5)

</details>

Les amplitudes les plus grandes sont à 3 et 4 cycles par longueur d'enregistrement, comme prévu, mais maintenant il y a des amplitudes réelles et imaginaires non nulles à *toutes* les fréquences. C'est le phénomène de *fuite*, et il est fondamental pour la transformée de Fourier et pour toutes les techniques d'analyse de données basées sur celle-ci.

In [ ]:
# Exercice
# appelez la fonction sinusoid et donnez-lui seulement l'argument cycles = 3.2

Maintenant, le signal est plus proche d'une fréquence de Fourier, et l'amplitude la plus grande est à cette fréquence la plus proche, 3 cycles par longueur d'enregistrement. La distribution des parties réelles et imaginaires de l'amplitude est plus compliquée que dans l'exemple précédent, mais montre toujours la symétrie attendue : $H_n = H^*_{-n} \:$.

Essayez une entrée complexe, une exponentielle complexe à 3 cycles par longueur d'enregistrement tournant dans le sens contraire des aiguilles d'une montre :

In [ ]:
# Exercice
# appelez la fonction sinusoid et donnez-lui seulement l'argument rotation = 'positive'

Et en tournant dans le sens des aiguilles d'une montre :

In [ ]:
# Exercice
# appelez la fonction sinusoid et donnez-lui seulement l'argument rotation = 'négative'

Remarquez la relation de phase attendue entre les parties réelles et imaginaires de l'entrée, et l'isolement de la transformée de Fourier à une seule amplitude non nulle, à une fréquence positive pour une rotation dans le sens contraire des aiguilles d'une montre, et à une fréquence négative pour une rotation dans le sens des aiguilles d'une montre.

### Transformation d'une impulsion

Nous avons vu que la TF d'une sinusoïde est fortement pointue - c'est-à-dire qu'elle est localisée dans l'espace des fréquences, même si une certaine énergie est répartie par fuite. Dans le cas idéal, où la sinusoïde échantillonnée a un nombre entier de cycles par longueur d'enregistrement, la TF est un seul pic (ou deux pics, si nous traitons séparément les fréquences positives et négatives). Que se passe-t-il lorsque le signal est pointu (hautement localisé) dans le *domaine du temps* ? Examinons la TF d'une impulsion de largeur et de position variables :

In [ ]:
def pulse(npts = 32, uplims=None, dt=1):
    """
    Génère et affiche un signal d'impulsion et sa transformation.
    """
    x1 = np.arange(npts, dtype=float) * dt
    y1 = np.zeros_like(x1)
    if uplims is None:
        upslice = slice(0, npts // 2)
    else:
        upslice = slice(*uplims)
    y1[upslice] = 1    
    show_transform(x1, y1)

Essayez une impulsion à l'origine - un seul point non nul :

In [ ]:
pulse(uplims=[0, 1])

Déplacez le point relevé vers la droite :

In [ ]:
# Exercice
# Appelez la fonction pulse mais donnez les uplims comme 1 et 2

Remarquez que bien que les transformations de ces deux signaux très similaires semblent assez différentes, elles ne diffèrent en réalité que par la phase ; dans les deux cas, l'amplitude complexe est uniforme avec la fréquence. *La localisation la plus extrême dans le domaine du temps produit l'uniformité la plus complète dans le domaine de la fréquence.*

Essayez maintenant une impulsion plus large :

In [ ]:
# Exercice
# Appelez la fonction pulse mais donnez les uplims comme 0 et 4

In [ ]:
# Exercice
# Appelez la fonction pulse mais donnez les uplims comme 0 et 8

In [ ]:
# Exercice
# Appelez la fonction pulse mais donnez les uplims comme 0 et 16

Avec une impulsion plus large - moins localisée dans l'espace - la plage de fréquences avec une grande amplitude se rétrécit.

### Transformation du bruit

Nous avons noté au début que la TF est particulièrement utile pour analyser les séries temporelles de type bruit, sans périodicité évidente. Dans de tels cas, nous sommes généralement intéressés à connaître le caractère spectral des données, souvent décrit par analogie avec le spectre visuel. Une série temporelle blanche est une série avec une variance approximativement uniforme par unité de fréquence ; un spectre rouge est pondéré vers les basses fréquences ; et un spectre bleu est pondéré vers les hautes fréquences.

In [ ]:
def noise(npts = 32, color='w', dt=1, repeatable=True):
    """
    Génère et affiche un signal de bruit et sa transformation.
    
    La couleur peut être 'r', 'w', ou 'b'.

    Pour expérimenter avec différents ensembles de nombres pseudo-aléatoires,
    définissez *repeatable* sur *False*.
    """
    if repeatable:
        np.random.seed(0)
    x1 = np.arange(npts, dtype=float) * dt
    y1 = np.random.randn(npts + 1)
    if color == 'w':
        y1 = y1[:npts]
    elif color == 'r':
        y1 = y1.cumsum()[:npts]
    elif color == 'b':
        y1 = np.diff(y1)
    y1 -= y1.mean()    
    show_transform(x1, y1)

Pour chacun des suivants, changez l'argument `repeatable` à `'False'` et exécutez la cellule à plusieurs reprises pour voir comment les résultats changent avec chaque réalisation du processus aléatoire.

In [ ]:
noise(color='r', repeatable=True)

In [ ]:
# Exercice
# Appelez la fonction noise mais donnez la couleur comme 'w' et réglable doit être défini sur True

In [ ]:
# Exercice
# Appelez la fonction noise mais donnez la couleur comme 'b'

Dans les trois exemples ci-dessus, passant du rouge au blanc au bleu, vous pouvez voir le changement dans les distributions d'amplitude, de l'accent sur les basses fréquences, à l'uniforme, à l'accent sur les hautes fréquences. Comme nous travaillons avec des échantillons aléatoires, les valeurs de la TF sont également aléatoires.

### Transformation d'une tendance

In [ ]:
def trend(npts = 32, dt=1):
    """
    Génère et affiche une tendance linéaire et sa transformation.
    """
    x1 = np.arange(npts, dtype=float) * dt
    y1 = np.arange(npts, dtype=float)
    y1 -= y1.mean()
    
    show_transform(x1, y1)

In [ ]:
trend()

## Transformation inverse

Mis à part l'erreur de troncature en virgule flottante, la TF inverse annule l'action de la TF. Cependant, nous pouvons mettre une étape au milieu pour atténuer ou supprimer l'énergie à certaines fréquences, filtrant ainsi le signal. Nous illustrerons cela ici en utilisant une coupure basse fréquence nette. L'impulsion sera le premier cobaye. L'effet de la coupure nette sera plus clair avec une série temporelle plus longue, d'où le choix de 256 points.

In [ ]:
def pulse_signal(npts = 256, uplims=None, dt=1):
    """
    Génère et renvoie un signal d'impulsion.
    """
    x1 = np.arange(npts, dtype=float) * dt
    y1 = np.zeros_like(x1)
    if uplims is None:
        upslice = slice(0, npts // 2)
    else:
        upslice = slice(*uplims)
    y1[upslice] = 1    
    return x1, y1

In [ ]:
def reconstruct(y, cutoff=None):
    """
    Reconstruit une série temporelle via l'ifft de sa fft, éventuellement en supprimant
    les fréquences à et au-dessus d'un seuil.

    cutoff est la fréquence la plus basse en cycles entiers par longueur d'enregistrement
    qui doit être omise dans la reconstruction.
    """
    y = np.asarray(y)
    npts = len(y)
    fy = fft(y)
    if cutoff is not None:
        if cutoff > 1:
            fy[cutoff:-(cutoff-1)] = 0
        else:
            raise ValueError("cutoff doit être supérieur à 1")
    yr = ifft(fy)
    if y.dtype.kind == 'c':
        return yr
    return yr.real

In [ ]:
def show_pulse_reconstruction(cutoffs, **kw):
    """
    cutoffs doit être une liste de fréquences de coupure en cycles par longueur d'enregistrement
    """
    x, y = pulse_signal(**kw)
    yrs = np.zeros((len(x), len(cutoffs)), dtype=float)
    for i, cutoff in enumerate(cutoffs): 
        yrs[:, i] = reconstruct(y, cutoff=cutoff)
    fig, ax = plt.subplots()
    line0 = ax.plot(x, y, color='k', lw=3)
    lines = ax.plot(x, yrs)
    linelist = line0 + lines
    labels = ["orig"]
    labels += ["%s" % c for c in cutoffs]
    ax.legend(linelist, labels)
    ax.grid(True)
    ax.set_xlabel('Temps')
    ax.set_title('Séries de Fourier tronquées')

In [ ]:
show_pulse_reconstruction([3, 5, 10, 20])

Remarquez que la reconstruction s'améliore à mesure que nous déplaçons la coupure vers des fréquences plus élevées, mais elle oscille toujours autour de sa cible. Y a-t-il une meilleure façon de filtrer en utilisant la FFT et l'IFFT ?

Au lieu d'utiliser une coupure nette, essayons de diminuer progressivement jusqu'à zéro.

In [ ]:
def reconstruct2(y, cutoff=None, taperfrac=0.25):
    """
    Reconstruit une série temporelle via l'ifft de sa fft, éventuellement en supprimant
    les fréquences à et au-dessus d'un seuil.

    cutoff est à peu près à mi-chemin de la pente de la diminution.
    """
    y = np.asarray(y)
    npts = len(y)
    fy = fft(y)
    if cutoff is not None:
        if cutoff > 1:
            w = np.ones((npts,), dtype=float)
            ntaper = int(round(taperfrac * cutoff))
            ntaper = min(cutoff - 2, ntaper)
            w[cutoff + ntaper:-(cutoff + ntaper - 1)] = 0
            w[cutoff - ntaper:cutoff + ntaper + 1] = np.linspace(1, 0, num=2 * ntaper + 1)
            w[-cutoff - ntaper:-cutoff + ntaper + 1] = np.linspace(0, 1, num=2 * ntaper + 1)
        else:
            raise ValueError("cutoff doit être supérieur à 1")
    yr = ifft(fy * w)
    if y.dtype.kind == 'c':
        return yr, w
    return yr.real, w

In [ ]:
def show_pulse_reconstruction2(cutoffs, taperfrac=0.25, **kw):
    """
    cutoffs doit être une liste de fréquences de coupure en cycles par longueur d'enregistrement
    """
    x, y = pulse_signal(**kw)
    yrs = np.zeros((len(x), len(cutoffs)), dtype=float)
    for i, cutoff in enumerate(cutoffs): 
        yrs[:, i] = reconstruct2(y, cutoff=cutoff, taperfrac=taperfrac)[0]
    fig, ax = plt.subplots()
    line0 = ax.plot(x, y, color='k', lw=3)
    lines = ax.plot(x, yrs)
    linelist = line0 + lines
    labels = ["orig"]
    labels += ["%s" % c for c in cutoffs]
    ax.legend(linelist, labels)
    ax.grid(True)
    ax.set_xlabel('Temps')
    ax.set_title('Séries de Fourier atténuées')

In [ ]:
show_pulse_reconstruction2([3, 5, 10, 20])

In [ ]:
show_pulse_reconstruction2([3, 5, 10, 20], taperfrac=0.75)

Nous avons utilisé un atténuation très grossière, mais elle aide toujours à supprimer les oscillations.

# Section 3
## Analyse spectrale
Nous avons l'habitude de penser à la lumière en termes de son contenu en couleur, et au son, surtout à la musique, en termes de son contenu en fréquence. Nous faisons de même avec les séries temporelles géophysiques - nous les décrivons, les décomposons ou les filtrons en fonction de la fréquence. Plus généralement, cela peut être étendu aux dimensions spatiales. Le spectre des vagues de gravité de surface de l'océan peut être décrit en termes de longueur d'onde ou de nombre d'onde, bien que cela introduise la complication supplémentaire que le nombre d'onde est un vecteur alors que la fréquence est un scalaire. Dans tous les cas, nous travaillons avec la façon dont l'énergie est répartie entre les composantes périodiques, et tant que nous ne considérons qu'une seule variable, nous nous soucions de l'énergie ou de l'amplitude, mais pas de la phase. C'est un point central : l'autospectre, qui est le sujet de ce cahier, *élimine les informations sur la phase*.

Note : dans tout ce qui suit, nous faisons des tracés rapides à la manière d'un script, sans chercher à ajouter les agréments habituels tels que les étiquettes d'axes et les légendes. Ces tracés sont destinés à votre expérimentation, et non comme des exemples de produits que l'on utiliserait pour d'autres fins que ces fins pédagogiques. Les échelles de temps sont en jours, les fréquences sont en cycles par jour, étant donné des données horaires. La densité spectrale de puissance est en amplitude carrée par cycle par jour.

In [ ]:
%matplotlib inline 
# substituez notebook par inline ci-dessus pour obtenir des graphiques interactifs
# graphiques en ligne

import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as ss

plt.rcParams['figure.dpi'] = 90

In [ ]:
def datafaker(nt, dt=1, freqs=None, color='w',
              amp=1, 
              complex=True,
              repeatable=True):
    """
    Génère de fausses données avec des sinusoides optionnels (tous de
    la même amplitude) et avec du bruit rouge, blanc ou bleu
    d'amplitude arbitraire.
    
    *nt* : nombre de points
    *dt* : incrément de temps en unités de temps arbitraires
    *freqs* : None, ou une séquence de fréquences en
        cycles par unité de temps. 
    *color* : 'r', 'w', 'b'
    *amp* : amplitude du bruit rouge, blanc ou bleu
    *complex* : True, False
    *repeatable* : True, False

    Retourne t, x
    """
    if repeatable:
        np.random.seed(1)    
    noise = np.random.randn(nt + 1) + 1j * np.random.randn(nt + 1)
    
    if color == 'r':
        noise = np.cumsum(noise) / 10 
        noise -= noise.mean()
    elif color == 'b':
        noise = np.diff(noise)
    noise = noise[:nt]
    x = amp * noise

    t = np.arange(nt, dtype=float) * dt
    
    for f in freqs:
        sinusoid = np.exp(2 * np.pi * 1j * f * t)
        x += sinusoid
    if not complex:
        x = np.real(x)
        
    return t, x

Essayons des données horaires, réelles (au lieu de complexes), avec une forte marée semi-diurne plus du bruit rouge.

In [ ]:
nt = 240

dt = 1/24 # 1 heure
tides = [24/12.42, 24/12]

t, h = datafaker(nt, dt=dt, freqs=tides, amp=1, 
                 color='r',
                 complex=False)
fig, ax = plt.subplots()
ax.plot(t, h)
ax.set_xlabel('days');

Le point de départ de l'autospectre est la transformée de Fourier discrète. Les amplitudes au carré - en éliminant la phase - donnent un périodogramme brut. Nous commençons avec une série réelle, donc nous n'avons pas besoin des fréquences positives et négatives. Nous pourrions utiliser `np.fft.rfft` qui ne calcule que les fréquences positives, mais comme nous travaillerons avec des entrées complexes plus tard, nous utiliserons plutôt `np.fft.fft` pour tout, et sacrifierons un peu d'efficacité.

In [ ]:
def spectrum1(h, dt=1):
    """
    Première tentative d'estimation spectrale : très brute.
    
    Renvoie les fréquences, le spectre de puissance, et
    la densité spectrale de puissance.
    Seules les fréquences positives entre (et non incluses)
    zéro et le Nyquist sont renvoyées.
    """
    nt = len(h)
    npositive = nt//2
    pslice = slice(1, npositive)
    freqs = np.fft.fftfreq(nt, d=dt)[pslice] 
    ft = np.fft.fft(h)[pslice]
    psraw = np.abs(ft) ** 2
    # Double pour tenir compte de l'énergie dans les fréquences négatives.
    psraw *= 2
    # Normalisation pour le spectre de puissance
    psraw /= nt**2
    # Convertir le spectre de puissance en densité spectrale de puissance
    psdraw = psraw * dt * nt  # nt * dt est la durée de l'enregistrement
    return freqs, psraw, psdraw

In [ ]:
# Choisissez deux longueurs de données en nombre d'échantillons. Ces dernières seront utilisées
# tout au long des exemples suivants. Elles doivent être assez grandes
# pour que les exemples fonctionnent bien.
n1 = 2400
n2 = 24000

dfkw = dict(dt=dt, freqs=tides, amp=1, color='r', complex=False)

t, h1 = datafaker(n1, **dfkw)
freqs1, ps1, psd1 = spectrum1(h1, dt=dt)

t, h2 = datafaker(n2, **dfkw)
freqs2, ps2, psd2 = spectrum1(h2, dt=dt)

fig, axs = plt.subplots(ncols=2, sharex=True)
axs[0].loglog(freqs1, psd1, 'r',
              freqs2, psd2, 'b', alpha=0.5)
axs[1].loglog(freqs1, ps1, 'r', 
              freqs2, ps2, 'b', alpha=0.5)
axs[0].set_title('Densité spectrale de puissance')
axs[1].set_title('Spectre de puissance')
axs[1].axis('tight')

Étant donné une série temporelle de $N$ points, $h_j$, et les $N$ coefficients de la Transformée de Fourier correspondants, $H_k$, alors le théorème de Parseval peut être écrit comme suit
$$\frac{1}{N}\sum_j |h_j|^2 = \frac{1}{N^2} \sum_k |H_k|^2.$$
C'est pourquoi, dans `spectrum1`, nous divisons chaque $|H_k|^2$ par $N^2$ pour obtenir le Spectre de Puissance (PS) --- de sorte que la valeur du PS à chaque fréquence sera la contribution de cette fréquence à la variance totale de la série temporelle.

Vérifiez cela pour notre exemple ; notez que `spectrum1` écarte la moyenne, en accord avec l'utilisation de la méthode `var` de la série temporelle. Cependant, il écarte également le Nyquist, nous perdons donc un tout petit peu de la variance.

In [ ]:
print('PS sum:   %.2f, %.2f' % (ps1.sum(), ps2.sum()))
print('Variance: %.2f, %.2f' % (h1.var(), h2.var()))
print('Differences: %g, %g' % (h1.var() - ps1.sum(),
                               h2.var() - ps2.sum()))

Remarquez que pour les pics de marée, le PS préserve l'amplitude tandis que le PSD préserve l'intégrale, mais pas l'amplitude. Pour le bruit de fond, leurs rôles sont inversés : le niveau spectral est préservé par le PSD, mais pas par le PS.

Pour obtenir des nombres stables dans les calculs ci-dessus, il faut des séries temporelles suffisamment longues, d'où le choix de 2400 et 24000 heures.

Maintenant, pourquoi obtenons-nous une intégrale PSD sur la bande de marée d'environ 1, et des valeurs maximales de PS de 0,5 ? Rappelez-vous que notre fausse marée est générée avec deux sinusoides d'amplitude unitaire. La variance contribuée par chacun est de 0,5, d'où la valeur maximale du PS. En intégrant sur la bande de marée, nous avons la somme des deux variances, donc 1.